## Validate the Gradient for Rest Geodesic Kappa 

In [ ]:
elastic_rods_dir = '../'
weaving_dir = '../../../Weaving/'
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from linkage_utils import order_segments_by_ribbons

# Reloadable libraries
import analysis_helper, importlib
importlib.reload(analysis_helper)
from analysis_helper import construct_elastic_rod_loop_from_rod_segments

### Use the sphere weaving model to get the rod test sample

In [ ]:
# Sphere Parameters
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [1, 10]
ISO_CS = [4, 4]
MODEL_PATH = osp.join(weaving_dir, 'models/equal_edge_atI.obj')
SUBDIVISION_RESOLUTION = 5

In [ ]:
with so():
    dodecahedron_linkage = elastic_rods.RodLinkage(MODEL_PATH, SUBDIVISION_RESOLUTION, False)
    driver=dodecahedron_linkage.centralJoint()
    dodecahedron_linkage.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, RIBBON_CS, stiffAxis=elastic_rods.StiffAxis.D1))
#     restlen_solve is broken until the design parameter refactoring is done.
    # elastic_rods.restlen_solve(dodecahedron_linkage)
    jdo = dodecahedron_linkage.dofOffsetForJoint(driver)
    fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
    elastic_rods.compute_equilibrium(dodecahedron_linkage, fixedVars=fixedVars)
    dodecahedron_view = linkage_vis.LinkageViewer(dodecahedron_linkage, width=1024, height=640)
    dodecahedron_view.setCameraParams(default_camera_parameters)
# dodecahedron_view.show()

In [ ]:
strips = order_segments_by_ribbons(dodecahedron_linkage)
new_rod, fixedVars = construct_elastic_rod_loop_from_rod_segments(dodecahedron_linkage, strips[0])
# Set the material of the new rod to be the same as previously.
new_rod.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, RIBBON_CS, stiffAxis=elastic_rods.StiffAxis.D1))
single_rod_view = linkage_vis.LinkageViewer(new_rod, width=1024, height=640)
with so():
    elastic_rods.compute_equilibrium(new_rod, fixedVars=fixedVars)
single_rod_view.setCameraParams(default_camera_parameters)
single_rod_view.show()

### Compute the difference between the gradient and the finite difference approximation

In [ ]:
new_rod.bendingEnergyType = elastic_rods.BendingEnergyType.Bergou2010

In [ ]:
# Gradient test
grad = new_rod.gradEnergyBend(variableDesignParameters = True)
curr_energy_type = elastic_rods.EnergyType.Bend
energy = new_rod.energy(energyType = curr_energy_type)
prevDof = new_rod.getExtendedDoFs()
experiment_steps = np.logspace(np.log10(1e-9), np.log10(1e-2), num = 100)
def experiment_in_diff_direction():
    random_direction = np.random.normal(0, 1, len(grad))
    direction = random_direction * (la.norm(np.array(grad)) / la.norm(random_direction))
    def get_difference_in_gradient(stepSize):
        new_rod.setExtendedDoFs(prevDof + stepSize * direction)
        energy_with_pos_step = new_rod.energy(curr_energy_type)

        new_rod.setExtendedDoFs(prevDof - stepSize * direction)
        energy_with_neg_step = new_rod.energy(curr_energy_type)

        fd_approx = (energy_with_pos_step - energy_with_neg_step) / (2 * stepSize)
        gradient_dot = np.dot(direction, grad)
        return abs(fd_approx - gradient_dot)

    epsilon_list = [get_difference_in_gradient(step_size) for step_size in experiment_steps]
    plt.plot(experiment_steps, epsilon_list)

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.set_xscale('log')
ax.set_yscale('log')
plt.xlabel('Decreasing Step Size')
plt.ylabel('Error')
plt.grid()
# ax.invert_xaxis()
for i in range(1):
    experiment_in_diff_direction()

plt.savefig('gradient_validation.png', dpi = 300)
new_rod.setExtendedDoFs(prevDof)